# Importation des données

In [117]:
import os
import string
import re
import math
from math import sqrt
import numpy as np
import random
import time
import pandas as pd
import nltk, re, pprint
#nltk.download('punkt')
from nltk import word_tokenize
random.seed(1)

import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import pickle
nb_tweets = 100000

os.chdir('/Users/alainquartierlatente/Desktop/Ensae/StatApp')

with open("data/corpus_trie.file", "rb") as f:
    corpus = pickle.load(f) 
ens_tweets = [phrase.split() for phrase in corpus]
len(ens_tweets)

1344097

On ne garde ici que les {{nb_tweets}} premiers tweets

In [122]:
phrases = ens_tweets[0:nb_tweets]
words = [item for sublist in phrases for item in sublist]
print(type(words))
vocabulary = set(words)
index_mot = {index: w for (index, w) in enumerate(vocabulary)}
mot_index = {w: index for (index, w) in enumerate(vocabulary)}
print("Nombre de mots :", len(words))
print("Taille du vocabulaire :", len(vocabulary))
fdist = nltk.FreqDist(words)

<class 'list'>
Nombre de mots : 1111355
Taille du vocabulaire : 34931


In [123]:
with open('data/input_%ik.file' % int(nb_tweets/1000), "rb") as f:
    input = pickle.load(f)
with open('data/output_%ik.file' % int(nb_tweets/1000), "rb") as f:
    output = pickle.load(f)
mat_finale = (input+output)/2

# 1. Similarité cosinus
Pour tester la similarité entre deux mots, on calcule la similarité cosinus entre leur représentation vectorielle :
$$
\frac{A \cdot B}{|| A || \cdot || B ||}
$$

In [124]:
mot_poids = {index_mot[index]: poids.detach().numpy() for (index, poids) in enumerate(mat_finale)}

def cos_distance(u, v):
    return (np.dot(u, v)  / (math.sqrt(np.dot(u, u)) *  (math.sqrt(np.dot(v, v)))))
def mot_plus_proche(word, n=10):
    word_distance = {}
    for mot in mot_poids:
        if mot != word:
            word_distance[mot] = (cos_distance(mot_poids[mot],(mot_poids[word])))
    word_distance = sorted(word_distance.items(), key=lambda t: t[1],reverse=True)
    return word_distance[0:n]
print(fdist.most_common(100))

[('nickname', 89991), ('de', 39547), ('le', 25200), ('est', 21065), ('je', 20346), ('les', 18506), ('c', 17675), ('j', 17373), ('la', 16886), ('à', 16590), ('pas', 16303), ('que', 12285), ('en', 12169), ('un', 11196), ('et', 10708), ('a', 10113), ('ça', 9796), ('ai', 9110), ('l', 8660), ('mais', 8316), ('il', 8295), ('pour', 7892), ('tu', 7453), ('d', 7379), ('on', 7049), ('😂', 6887), ('une', 6871), ('me', 6226), ('qui', 5488), ('ce', 5437), ('t', 5269), ('😭', 5106), ('dans', 4929), ('plus', 4898), ('sur', 4836), ('mon', 4530), ('moi', 4479), ('suis', 4457), ('qu', 4358), ('y', 4166), ('avec', 3970), ('fait', 3957), ('bien', 3907), ('trop', 3823), ('vous', 3815), ('ma', 3668), ('si', 3622), ('tout', 3309), ('m', 3236), ('elle', 3205), ('va', 3168), ('faire', 3076), ('même', 3048), ('merci', 2790), ('toi', 2771), ('ne', 2766), ('bon', 2712), ('quand', 2694), ('mdr', 2607), ('oui', 2521), ('ils', 2496), ('comme', 2482), ('non', 2463), ('n', 2449), ('se', 2358), ('là', 2271), ('être', 226

In [157]:
mots = ["tweet", "bonjour", "wesh"]
for mot in mots:
    if mot in mot_index:
        print('Pour le mot "%s"' %mot + " (%i apparitions), les mots les plus proches sont : " %fdist[mot])
        print(mot_plus_proche(mot))
        print("\n")

Pour le mot "tweet" (444 apparitions), les mots les plus proches sont : 
[('sciemment', 0.7796510320842351), ('larbin', 0.7743906922762018), ('goudi', 0.7682426641322974), ('pénal', 0.7314700136059535), ('pe', 0.728211820622303), ('asiatique', 0.7073907839022556), ('kiwi', 0.7007462689075674), ('little', 0.6944771346910209), ('grotesque', 0.6916143022805173), ('was', 0.689915584522579)]


Pour le mot "bonjour" (619 apparitions), les mots les plus proches sont : 
[('ix', 0.8152861253987886), ('standards', 0.7849611194127664), ('maupassant', 0.772227569018608), ('kangourou', 0.7425559660443132), ('damp', 0.7402806470915697), ('calin', 0.7198772753357117), ('riddim', 0.6864688830711003), ('68', 0.6837049816321749), ('lunes', 0.6826879511610323), ('👧', 0.6786617196604566)]


Pour le mot "wesh" (119 apparitions), les mots les plus proches sont : 
[('mathilde', 0.8333691344124079), ('footballer', 0.7587564416964777), ('🎥', 0.7494822228275589), ('tomorrowland', 0.7323953999825153), ('pistolet

# 2. Evaluation par le "human judgement"

In [158]:
from scipy.stats import spearmanr
df = pd.read_csv('data_bis/word_similarity.csv', sep=";")
df

,word1,word2,corr
0,corde,sourire,0.00
1,midi,ficelle,0.00
2,coq,périple,0.06
3,fruit,fournaise,0.11
4,autographe,rivage,0.00
...,...,...,...
60,coussin,oreiller,3.00
61,cimetière,cimetière,4.00
62,automobile,auto,3.94
63,joyau,bijou,3.22


In [159]:
def distance_mots(word1,word2):
    if word1 in mot_poids.keys() and word2 in mot_poids.keys():
        word_distance = (cos_distance(mot_poids[word1],(mot_poids[word2])))
    else:
        word_distance = float('nan')
    return word_distance
distance_mots_v = np.vectorize(distance_mots)
df["corr_word2vec"] = distance_mots_v(df["word1"],df["word2"])
print(len(df))
df = df.dropna()
print(len(df))
df

65
45


,word1,word2,corr,corr_word2vec
0,corde,sourire,0.00,-0.190808
1,midi,ficelle,0.00,0.086974
2,coq,périple,0.06,-0.017321
5,automobile,sorcier,0.00,0.135497
7,grimace,instrument,0.00,-0.033913
8,refuge,fruit,0.00,-0.151045
9,refuge,moine,0.22,0.139041
10,cimetière,asylum,0.22,-0.002258
11,garçon,coq,0.44,-0.059926
12,verre,magicien,0.06,-0.246610


In [160]:
#On fait des tests à 5 %
alpha = 0.05
corr, p_value = spearmanr(df["corr"], df["corr_word2vec"])
if p_value > alpha:
    print('Le résultat de word2vec est différent de celui du human judgement (non rejet de H0 = non corrélation) p=%.3f' % p_value,'/ Valeur de la corrélation : %.3f'% corr)
else:
    print('Le résultat de word2vec est semblable celui du human judgement (rejet de H0 = non corrélation) p=%.3f' % p_value,'/ Valeur de la corrélation : %.3f'% corr)

Le résultat de word2vec est différent de celui du human judgement (non rejet de H0 = non corrélation) p=0.297 / Valeur de la corrélation : 0.159
